# Getting Started with the `mpc_api` Python Package

#### This tutorial demonstrates how to use the `mpc_api` package to interact with the Minor Planet Center's REST APIs.

The [`mpc_api`](https://github.com/Smithsonian/mpc-public/tree/main/mpc-api) package provides a single `MPCClient` class that wraps all of the MPC's public APIs into convenient Python methods. Instead of writing raw HTTP requests, you can call methods like `mpc.identify("Ceres")` or `mpc.get_orbit("Bennu")`.

**Features:**
 - Single `MPCClient` class covering all MPC APIs
 - Input validation with clear error messages
 - Optional pandas DataFrame output for analysis-friendly results
 - Custom exception hierarchy for structured error handling

**Installation:**

```bash
pip install mpc-api              # core (returns dicts)
pip install mpc-api[dataframe]   # adds pandas DataFrame methods
```

**Individual API tutorials** that show both raw `requests` and `mpc_api` approaches are available at:
 - https://docs.minorplanetcenter.net/tutorials/api_tutorials/

**Documentation** regarding the individual APIs is available at:
 - https://docs.minorplanetcenter.net/mpc-ops-docs/apis/

# Install Required Packages

Run the cell below to ensure the `mpc_api` package and its dependencies are installed in the current Python environment. If you already have them installed, this cell will complete quickly.

In [ ]:
# Install required packages (run this cell if packages are not already installed)
import subprocess
import sys

# Install mpc-api with the dataframe extra (includes pandas)
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "mpc-api[dataframe]"])
print("Installation complete.")

# Import Packages

Here we import the `mpc_api` package along with standard libraries for display.

In [2]:
from mpc_api import MPCClient
import json

# Create a client instance (used for all API calls)
mpc = MPCClient()
print(mpc)

MPCClient()


# Designation Identifier

The `identify()` method resolves object names, numbers, and provisional designations into canonical MPC identifiers.

It accepts a single string or a list of strings.

In [3]:
# Single object lookup
result = mpc.identify("Sedna")
print(json.dumps(result, indent=4))

{
    "Sedna": {
        "citation": "(90377) Sedna = 2003 VB12<br><br>Sedna is the Inuit goddess of the sea and the mother of all sea creatures. She rewards the people of the land with food from the sea. Without her blessing, hunts fail and the people starve. She is thus one of the most important figures in Inuit legend. ",
        "disambiguation_list": null,
        "dual_status_info": null,
        "found": 1,
        "iau_designation": "(90377)",
        "name": "Sedna",
        "object_type": [
            "Minor Planet",
            0
        ],
        "orbfit_name": "90377",
        "packed_permid": "90377",
        "packed_primary_provisional_designation": "K03V12B",
        "packed_secondary_provisional_designations": [],
        "permid": "90377",
        "unpacked_primary_provisional_designation": "2003 VB12",
        "unpacked_secondary_provisional_designations": []
    }
}


In [4]:
# Multi-object lookup in a single call
result = mpc.identify(["Ceres", "2023 BU", "Apophis"])

for name, info in result.items():
    print(f"{name:12s}  permid={str(info.get('permid', 'N/A')):>8s}  type={info['object_type'][0]}")

2023 BU       permid=    None  type=Minor Planet
Apophis       permid=   99942  type=Minor Planet
Ceres         permid=       1  type=Minor Planet


# Orbits

The `get_orbit()` method returns orbital elements in the `mpc_orb` JSON format. It extracts the orbit dict directly, returning `None` when the object is not found.

In [5]:
# Retrieve orbital elements for Bennu
orbit = mpc.get_orbit("Bennu")

if orbit:
    desig = orbit["designation_data"]
    print(f"Object:      {desig.get('permid', 'N/A')}")
    print(f"Designation: {desig.get('unpacked_primary_provisional_designation', 'N/A')}")
    print(f"H magnitude: {orbit['magnitude_data']['H']}")
    
    # Print cometarian orbital elements
    com = orbit["COM"]
    print("\nCometarian orbital elements:")
    for name, val, unc in zip(com["coefficient_names"], com["coefficient_values"], com["coefficient_uncertainties"]):
        print(f"  {name:>10s} = {val:>15.8f}  +/- {unc}")

Object:      101955
Designation: 1999 RQ36
H magnitude: 20.684

Cometarian orbital elements:
           q =      0.89659064  +/- 9.1402e-09
           e =      0.20373108  +/- 8.17369e-09
           i =      6.03290398  +/- 1.22908e-06
        node =      1.97241281  +/- 2.55606e-06
     argperi =     66.39437769  +/- 3.04048e-06
   peri_time =  60675.73787173  +/- 1.90921e-06


In [6]:
# A non-existent object returns None (not an error)
result = mpc.get_orbit("NotARealAsteroid")
print(f"Result: {result}")

Result: None


# Observations

The `get_observations()` method retrieves observation data in various formats. For data analysis, the `get_observations_df()` convenience method returns a pandas DataFrame directly.

In [7]:
# Get observations in the classic 80-column format
result = mpc.get_observations("2023 BU", output_format="OBS80")

lines = result["OBS80"].strip().split("\n")
print(f"Total observations: {len(lines)}")
print("\nFirst 3:")
for line in lines[:3]:
    print(f"  {line}")

Total observations: 1784

First 3:
  K23B00U  C2023 01 21.34299109 50 04.192+43 47 13.29         21.19RU~6CJqI41
       K23B00U  C2023 01 21.44005909 49 14.025+43 47 08.78         20.15RU~6CJqI41
       K23B00U*KC2023 01 21.99514 09 48 47.36 +43 41 03.0          19.5 GX~6CJqL51


In [8]:
# Get observations as a pandas DataFrame (requires pandas)
df = mpc.get_observations_df("Bennu")

print(f"Shape: {df.shape[0]} observations x {df.shape[1]} columns")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst 5 rows (selected columns):")
display_cols = [c for c in ["permid", "provid", "obstime", "ra", "dec", "mag", "stn"] if c in df.columns]
print(df[display_cols].head())

Shape: 603 observations x 76 columns

Columns: ['Obstype', 'artsat', 'astcat', 'band', 'com', 'ctr', 'dec', 'decstar', 'delay', 'deltadec', 'deltara', 'deprecated', 'disc', 'dist', 'doppler', 'exp', 'fltr', 'frq', 'localuse', 'logsnr', 'mag', 'mode', 'notes', 'nstars', 'nucmag', 'obscenter', 'obsid', 'obssubid', 'obstime', 'pa', 'permid', 'photap', 'photcat', 'pos1', 'pos2', 'pos3', 'poscov11', 'poscov12', 'poscov13', 'poscov22', 'poscov23', 'poscov33', 'precdec', 'precra', 'prectime', 'prog', 'provid', 'ra', 'rastar', 'rcv', 'ref', 'remarks', 'rmscorr', 'rmsdec', 'rmsdelay', 'rmsdist', 'rmsdoppler', 'rmsfit', 'rmsmag', 'rmspa', 'rmsra', 'rmstime', 'seeing', 'shapeocc', 'stn', 'subfmt', 'subfrm', 'sys', 'trkid', 'trkmpc', 'trksub', 'trx', 'unctime', 'vel1', 'vel2', 'vel3']

First 5 rows (selected columns):
   permid     provid                   obstime        ra        dec   mag  stn
0  101955  1999 RQ36  1999-09-11T09:44:59.136Z  24.47875  -27.07431  15.1  704
1  101955  1999 RQ36  19

# Observatory Codes

Look up observatory information by code, retrieve all observatories as a DataFrame, or search by name.

In [9]:
# Single observatory lookup
obs = mpc.get_observatory("F51")
print(f"Code: {obs['obscode']}")
print(f"Name: {obs['name']}")
print(f"Type: {obs['observations_type']}")

Code: F51
Name: Pan-STARRS 1, Haleakala
Type: optical


In [10]:
# Search observatories by name (returns a DataFrame)
results = mpc.search_observatories("Palomar")
print(results[["obscode", "name", "observations_type"]])

    obscode                         name observations_type
261     261         Palomar Mountain-DSS           optical
644     644        Palomar Mountain/NEAT           optical
675     675             Palomar Mountain           optical
I41     I41        Palomar Mountain--ZTF           optical
K10     K10  Micro Palomar, Reilhanette            optical


In [11]:
# Get all observatories as a DataFrame and analyze
all_obs = mpc.get_all_observatories_df()
print(f"Total observatories: {len(all_obs)}")

print("\nBy type:")
print(all_obs["observations_type"].value_counts())

Total observatories: 2681

By type:
observations_type
optical        2640
satellite        22
radar            15
occultation       2
roving            2
Name: count, dtype: int64


# MPECs (Minor Planet Electronic Circulars)

Search for MPECs by object designation or wildcard pattern. The `get_discovery_mpec()` convenience method returns the earliest MPEC for an object.

In [12]:
# Search for MPECs about Apophis
result = mpc.get_mpecs("Apophis")
mpecs = result.get("Apophis", [])

print(f"Found {len(mpecs)} MPECs for Apophis")
print("\nMost recent 3:")
for mpec in mpecs[:3]:
    print(f"  {mpec['fullname']:12s}  {mpec['pubdate'][:10]}  {mpec['title'][:50]}")

Found 9 MPECs for Apophis

Most recent 3:
  2004-Y25      Mon, 20 De  2004 MN4
  2004-Y60      Sat, 25 De  2004 MN4
  2004-Y63      Sun, 26 De  2004 MN4


In [13]:
# Get the discovery MPEC for an object
discovery = mpc.get_discovery_mpec("`Oumuamua")
if discovery:
    print(f"Discovery MPEC for `Oumuamua:")
    print(f"  MPEC:      {discovery['fullname']}")
    print(f"  Title:     {discovery['title']}")
    print(f"  Published: {discovery['pubdate']}")
    print(f"  Link:      {discovery['link']}")

Discovery MPEC for `Oumuamua:
  MPEC:      2017-V38
  Title:     `Oumuamua
  Published: Fri, 10 Nov 2017 19:15:00 GMT
  Link:      https://www.minorplanetcenter.net/mpec/K17/K17V38.html


# Check Near-Duplicates (CND)

Check whether observations already exist in the MPC database. Useful before submitting observations to avoid duplicates.

In [14]:
# Check a single observation in MPC 80-column format
obs = "f9671         C2020 02 21.46921410 05 10.27 +04 52 25.7          19.37oV~3n2UT08"

results = mpc.check_near_duplicates(obs)
print(json.dumps(results, indent=4))

{
    "f9671         C2020 02 21.46921410 05 10.27 +04 52 25.7          19.37oV~3n2UT08": [
        {
            "angle_separation_arcsec": 0.021,
            "obs80": "f9671         C2020 02 21.46921410 05 10.27 +04 52 25.7          19.37oV~3n2UT08",
            "time_separation_s": 0.0
        }
    ]
}


In [15]:
# Count duplicates (convenient for batch checking)
observations = [
    "f9671         C2020 02 21.46921410 05 10.27 +04 52 25.7          19.37oV~3n2UT08",
    "     abc123   C2020 03 23.46921410 05 13.27 +04 52 25.9          19.37oV     T08",
]

counts = mpc.count_near_duplicates(observations)
for obs_line, count in counts.items():
    status = f"{count} match(es)" if count > 0 else "no matches"
    print(f"  {obs_line[:40]}...  -> {status}")

       abc123   C2020 03 23.46921410 05 13...  -> no matches
  f9671         C2020 02 21.46921410 05 10...  -> 1 match(es)


# Submission Status

Check whether a previously submitted observation file was accepted or rejected by the MPC pipeline.

In [16]:
# Check status of a known accepted submission
status = mpc.get_submission_status("2026-01-01T00:05:07.453_0000BhCE")
print(json.dumps(status, indent=4))

{
    "accepted": true,
    "pipeline_entry_time": "2026-01-01T00:06:00.696565+00:00",
    "fault_events": []
}


# Submitting Observations

The `submit_xml()` and `submit_psv()` methods submit ADES-formatted observation files to the MPC. By default they use the **test** endpoint (`test=True`) so you can safely experiment.

You can pass a file path or raw content as a string.

In [17]:
import tempfile, os, atexit, requests

# Download sample XML file from the MPC
sample_url = "https://www.minorplanetcenter.net/media/ades/goodsubmit.xml.txt"
with tempfile.NamedTemporaryFile(mode='w', suffix='.xml', delete=False) as f:
    f.write(requests.get(sample_url).text)
    sample_xml_path = f.name
    atexit.register(lambda p=f.name: os.unlink(p) if os.path.exists(p) else None)

print(f"Sample XML saved to: {sample_xml_path}")

# Submit to test endpoint
result = mpc.submit_xml(
    sample_xml_path,
    ack="mpc_api tutorial",
    ac2="my@email.adr",
    obj_type="NEO",
    test=True,
)
print(f"Status: {result['status_code']}")
print(f"Response: {result['message']}")

Sample XML saved to: /var/folders/67/j23cbc8x5r3b_1cy48v0rf4m0000gq/T/tmpxq73um96.xml
Status: 200
Response: [mpc_api tutorial].  Submission ID is 2026-02-28T18:30:57.574_00000Ii8



# Action Codes

Request retrieval of an action code for a previous submission. The code is **emailed** to the original submitter (not returned in the response).

```python
# Replace with your actual submission ID
result = mpc.request_action_code("2026-01-01T00:05:07.453_0000BhCE")
print(result)
```

# Error Handling

The `mpc_api` package provides a custom exception hierarchy for structured error handling:

```
MPCAPIError (base)
  +-- MPCRequestError        # Network/timeout failures
  +-- MPCResponseError       # Non-2xx HTTP status codes
  |     +-- MPCNotFoundError # 404 specifically
  +-- MPCValidationError     # Local input validation failures
```

In [18]:
from mpc_api import MPCValidationError, MPCNotFoundError

# Input validation catches bad arguments before making an API call
try:
    mpc.get_observations("Bennu", output_format="INVALID_FORMAT")
except MPCValidationError as e:
    print(f"Validation error: {e}")

# CND parameter range validation
try:
    mpc.check_near_duplicates("some obs", time_separation_s=999)
except MPCValidationError as e:
    print(f"Validation error: {e}")

Validation error: Invalid output_format 'INVALID_FORMAT'. Must be one of {'OBS_DF', 'ADES_DF', 'OBS80', 'XML'}
Validation error: time_separation_s must be between 0 and 60


In [19]:
# MPCNotFoundError is raised for 404 responses
try:
    mpc.get_submission_status("2000-01-01T00:00:00.000_0000FAKE")
except MPCNotFoundError as e:
    print(f"Not found (expected): {e}")

Not found (expected): Not found (404): https://data.minorplanetcenter.net/api/submission-status


# Summary

The `mpc_api` package provides a convenient Python interface to all MPC public APIs.

**Key methods:**

| Method | Description |
|--------|-------------|
| `identify()` | Resolve object designations |
| `get_orbit()` | Get orbital elements (mpc_orb format) |
| `get_observations()` / `get_observations_df()` | Retrieve observations |
| `get_neocp_observations()` / `get_neocp_observations_df()` | NEOCP observations |
| `get_observatory()` / `get_all_observatories_df()` / `search_observatories()` | Observatory data |
| `get_mpecs()` / `get_discovery_mpec()` | MPEC search |
| `check_near_duplicates()` / `count_near_duplicates()` | Duplicate checking |
| `get_submission_status()` | Check submission status |
| `submit_xml()` / `submit_psv()` | Submit observations |
| `request_action_code()` | Retrieve action codes |

**Installation:** `pip install mpc-api` (or `pip install mpc-api[dataframe]` for DataFrame support)

**Source code:** https://github.com/Smithsonian/mpc-public/tree/main/mpc-api

**Individual API tutorials:** https://docs.minorplanetcenter.net/tutorials/api_tutorials/

For questions or feedback, contact the MPC via the [Jira Helpdesk](https://mpc-service.atlassian.net/servicedesk/customer/portal/13/create/148).